

# Building a healthcare chatbot with Mixtral 8x7b, Haystack, and PubMed


*notebook by Tilde Thurium:
 [Mastodon](https://tech.lgbt/@annthurium) || [Twitter](https://twitter.com/annthurium) || [LinkedIn](https://www.linkedin.com/in/annthurium/)*

##Introduction
**📚 Check out the [Building a healthcare chatbot with Mixtral 8x7b, Haystack, and PubMed](https://haystack.deepset.ai/blog/mixtral-8x7b-healthcare-chatbot) article for a detailed run through of this example.**

**Prerequisites:**

*   [HuggingFace Access Token](https://huggingface.co/settings/tokens)

## Installing Haystack

To start, let's install the latest release of Haystack with `pip`, as well as any other libraries we're going to need:

In [ ]:
%%bash

pip install haystack-ai
pip install pymed
pip install huggingface_hub

This asks for keys the notebook needs.

In [ ]:
from getpass import getpass

huggingface_token = getpass("Enter your Hugging Face api token:")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# PubMed Fetcher

PubMed is the best source of up to date medical research. Now we are going to write our own custom class to pull scientific papers from PubMed that are relevant to the query at hand.

The PubMed sdk basically just wraps the PubMed API so it's easier to query.


In [ ]:
# pymed → A library for querying PubMed articles.
# List → Used for type hinting to specify lists.
# haystack.component → Defines reusable Haystack components.
# haystack.Document → A class that stores the text content and metadata of a document.
from pymed import PubMed
from typing import List
from haystack import component
from haystack import Document

pubmed = PubMed(tool="Haystack2.0Prototype", email="tilde.thurium@deepset.ai")

def documentize(article):
  return Document(content=article.abstract, meta={'title': article.title, 'keywords': article.keywords})

@component
class PubMedFetcher():

  @component.output_types(articles=List[Document])
  def run(self, queries: list[str]):
    cleaned_queries = queries[0].strip().split('\n')

    articles = []
    try:
      for query in cleaned_queries:
        response = pubmed.query(query, max_results = 1)
        documents = [documentize(article) for article in response]
        articles.extend(documents)
    except Exception as e:
        print(e)
        print(f"Couldn't fetch articles for queries: {queries}" )
    results = {'articles': articles}
    return results

Now we add our `PubmedFetcher` into a pipeline.

In [ ]:
from haystack.components.generators import HuggingFaceTGIGenerator
from haystack.utils import Secret


keyword_llm = HuggingFaceTGIGenerator(model="mistralai/Mixtral-8x7B-Instruct-v0.1", token=Secret.from_token(huggingface_token))
keyword_llm.warm_up()

llm = HuggingFaceTGIGenerator(model="mistralai/Mixtral-8x7B-Instruct-v0.1", token=Secret.from_token(huggingface_token))
llm.warm_up()

In [ ]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder

keyword_prompt_template = """
Your task is to convert the following question into 3 keywords that can be used to find relevant medical research papers on PubMed.
Here is an examples:
question: "What are the latest treatments for major depressive disorder?"
keywords:
Antidepressive Agents
Depressive Disorder, Major
Treatment-Resistant depression
---
question: {{ question }}
keywords:
"""

prompt_template = """
Answer the question truthfully based on the given documents.
If the documents don't contain an answer, use your existing knowledge base.

q: {{ question }}
Articles:
{% for article in articles %}
  {{article.content}}
  keywords: {{article.meta['keywords']}}
  title: {{article.meta['title']}}
{% endfor %}

"""
keyword_prompt_builder = PromptBuilder(template=keyword_prompt_template)

prompt_builder = PromptBuilder(template=prompt_template)
fetcher = PubMedFetcher()

pipe = Pipeline()

pipe.add_component("keyword_prompt_builder", keyword_prompt_builder)
pipe.add_component("keyword_llm", keyword_llm)
pipe.add_component("pubmed_fetcher", fetcher)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)

pipe.connect("keyword_prompt_builder.prompt", "keyword_llm.prompt")
pipe.connect("keyword_llm.replies", "pubmed_fetcher.queries")

pipe.connect("pubmed_fetcher.articles", "prompt_builder.articles")
pipe.connect("prompt_builder.prompt", "llm.prompt")



While we're at it, let's make an `ask` method to wrap our query fetching. This method makes it easy to pull the query response out of the results and highlighting the answer in blue.

In [ ]:
from IPython.display import display, HTML

def ask(question):
  output = pipe.run(data={"keyword_prompt_builder":{"question":question},
                          "prompt_builder":{"question": question},
                          "llm":{"generation_kwargs": {"max_new_tokens": 500}}})
  print(question)
  print(output['llm']['replies'][0])
  # display(HTML(f'<div style="color: blue">{output["llm"]['replies'][0]}</div>'))


Give it a try!

In [ ]:
ask("How are mRNA vaccines being used for cancer treatment?")

**What's next**

In this tutorial, you learned to make a basic chatbot using a custom Haystack Retriever that pulls data from PubMed. In order to make this chatbot fancier, you could make improvements like:


*   Adding additional data sources
*   Giving the bot a name or personality
*   Making it a [stateful agent so it remembers past queries](https://docs.haystack.deepset.ai/docs/agent#conversational-agent-memory)

To see what else is possible with Haystack, you can [browse these tutorials](https://haystack.deepset.ai/tutorials) or check us out on [GitHub](https://github.com/deepset-ai/haystack). Thanks for reading!




